In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(tf.__version__)
print(gpus)

2.4.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import SimpleITK as sitk
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers,models,optimizers
import os, sys
from PIL import Image

In [3]:
img_file_dir=r'C:\Users\User\Desktop\hong\ICTS\train\img'
lab_file_dir=r'C:\Users\User\Desktop\hong\ICTS\train\lab'

IMG_SIZE=128

def ReFileName(dirPath):
    listdir=os.listdir(dirPath)
    img_list=[]
    for file in listdir:
        if os.path.isfile(os.path.join(dirPath, file)) == True:
            c= os.path.basename(file)
            name = dirPath + '\\' + c
            image=Image.open(name).resize((IMG_SIZE, IMG_SIZE))
            img_list.append(np.asarray(image))
    return img_list


In [4]:
img_list=ReFileName(img_file_dir)

In [5]:
lab_list=ReFileName(lab_file_dir)

In [6]:
image_numpy=np.asarray(img_list).astype('float32')/255.0
print(image_numpy.shape)
image_numpy=image_numpy.reshape(-1,IMG_SIZE,IMG_SIZE,1)
print(image_numpy.shape)
print('----------------')
lab_numpy=np.asarray(lab_list)
print(lab_numpy.shape)
lab_numpy=lab_numpy.reshape(-1,IMG_SIZE,IMG_SIZE,1)
print(lab_numpy.shape)


from sklearn.model_selection import train_test_split
train_data, val_data, train_label, val_label = train_test_split(image_numpy, lab_numpy, test_size=0.3, random_state=0)

print(train_data.shape)
print(train_label.shape)
print(val_data.shape)
print(val_label.shape)

(23512, 128, 128)
(23512, 128, 128, 1)
----------------
(23512, 128, 128)
(23512, 128, 128, 1)
(16458, 128, 128, 1)
(16458, 128, 128, 1)
(7054, 128, 128, 1)
(7054, 128, 128, 1)


In [7]:
# 2D
def create_model(input_size = (512,512,1)):
    l2 = tf.keras.regularizers.l2(0.01)
    
    inputs = layers.Input(input_size)
    conv1 = layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = layers.Dropout(0.5)(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)
    
    bn=layers.BatchNormalization()(pool4)
    
    conv5 = layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2 )(bn)
    conv5 = layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2)(conv5)
    drop5 = layers.Dropout(0.5)(conv5)

    up6 = layers.Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(drop5))
    merge6 = layers.concatenate([drop4,up6], axis = 3)
    conv6 = layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = layers.Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv6))
    merge7 = layers.concatenate([conv3,up7], axis = 3)
    conv7 = layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = layers.Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv7))
    merge8 = layers.concatenate([conv2,up8], axis = 3)
    conv8 = layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = layers.Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv8))
    merge9 = layers.concatenate([conv1,up9], axis = 3)
    conv9 = layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = layers.Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = layers.Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = tf.keras.models.Model(inputs=inputs, outputs=conv10)
    
    model.summary()
    return model

In [8]:
U_model=create_model((IMG_SIZE,IMG_SIZE,1))

metrics_list=[
tf.keras.metrics.Precision(name='precision'),
tf.keras.metrics.Recall(name='recall'),
tf.keras.metrics.BinaryAccuracy(name='accuracy'),
tf.keras.metrics.FalseNegatives(name='FN'),
tf.keras.metrics.FalsePositives(name='FP'),
tf.keras.metrics.TrueNegatives(name='TN'),
tf.keras.metrics.TruePositives(name='TP'),
tf.keras.metrics.AUC(name='ROC'),
]

adam=tf.keras.optimizers.SGD(learning_rate=0.05,momentum=0.5)
model_loss=tf.keras.losses.BinaryCrossentropy()


U_model.compile(optimizer=adam,
              loss=model_loss,
              metrics=metrics_list)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 64, 64, 64)   0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
# train_data, val_data, train_data, val_label 

callbacks_list=[
        tf.keras.callbacks.EarlyStopping(
          monitor='val_precision',mode='max', patience=10 ,restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath='C:/Users/User/Desktop/hong/ICTS/unet_06',
            monitor='val_precision',
            save_best_only=True,
        )
       ]


history=U_model.fit(train_data,train_data,
                    validation_data=(val_data,val_label),
                    epochs=25,
                    batch_size=16,
                    callbacks=callbacks_list)

Epoch 1/25
1029/1029 [==============================] - 179s 168ms/step - loss: 20.1075 - precision: 0.8952 - recall: 0.0321 - accuracy: 0.6577 - FN: 44303177.7544 - FP: 43851.9903 - TN: 89052944.6340 - TP: 1734838.0194 - ROC: 0.9196 - val_loss: 1.2586 - val_precision: 0.0484 - val_recall: 0.1836 - val_accuracy: 0.9794 - val_FN: 422486.0000 - val_FP: 1868349.0000 - val_TN: 113186776.0000 - val_TP: 95031.0000 - val_ROC: 0.8922
INFO:tensorflow:Assets written to: C:/Users/User/Desktop/hong/ICTS\unet_06\assets
Epoch 2/25
1029/1029 [==============================] - 171s 166ms/step - loss: 0.4761 - precision: 1.0000 - recall: 0.0511 - accuracy: 0.6589 - FN: 43768592.0204 - FP: 0.0000e+00 - TN: 89014538.1282 - TP: 2351639.7689 - ROC: 0.9785 - val_loss: 0.5952 - val_precision: 0.0488 - val_recall: 0.1936 - val_accuracy: 0.9786 - val_FN: 417325.0000 - val_FP: 1954496.0000 - val_TN: 113100688.0000 - val_TP: 100192.0000 - val_ROC: 0.8918
Epoch 3/25
1029/1029 [==============================] - 17

In [ ]:
U_model.save('unet_06.h5')

In [ ]:
def train_history_graphic( history       # 資料集合
                         , history_key1  # 資料集合裡面的來源 1 (有 loss, acc, val_loss, val_acc 四種)
                         , history_key2  # 資料集合裡面的來源 2 (有 loss, acc, val_loss, val_acc 四種)
                         , y_label       # Y 軸標籤文字
                         , index=0
                         ) :
    plt.figure(figsize=(9,9))
    font_size=20
    # 資料來源 1
    plt.plot( history.history[history_key1][index:] )
    # 資料來源 2
    plt.plot( history.history[history_key2][index:] )
    # 標題
    plt.title( 'train history',fontsize=font_size )
    # X 軸標籤文字
    plt.xlabel( 'epochs',fontsize=font_size )
    # Y 軸標籤文字
    plt.ylabel( y_label,fontsize=font_size )
    plt.xticks(fontsize=font_size)
    plt.yticks(fontsize=font_size)
    
    # 設定圖例
    # (參數 1 為圖例說明, 有幾個資料來源, 就對應幾個圖例說明)
    # (參數 2 為圖例位置, upper 為上面, lower 為下面, left 為左邊, right 為右邊)
    if 'loss' is history_key1:
        plt.legend( ['train', 'validate']
               , loc = 'upper right'
                ,fontsize=font_size)
    else :
        plt.legend( ['train', 'validate']
       , loc = 'lower right'
        ,fontsize=font_size)
    # 顯示畫布 filename
#     plot.savefig(r'D:\_YUN\26_DecisionAnalysis\A0510\ECG\img\{}_{}.png'.format(filename,history_key1))
    plt.show()

#############################################
    
metrics_name_list=['loss','precision','recall','accuracy']
for metrics in metrics_name_list :
  train_history_graphic( history, metrics, f'val_{metrics}', metrics, 2 )

# load_model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib 
import tensorflow as tf
from tensorflow import keras
import os, sys
from PIL import Image
import zipfile
from datetime import datetime

IMG_SIZE=128
model_name='unet_02'
test_model = keras.models.load_model(r'C:\Users\User\Desktop\hong\ICTS\{}'.format(model_name))

In [ ]:
# 建立 預測存放 predict檔案 的資料夾


dt = datetime.now().strftime('%m%d%H%M')
predict_dir=r'C:\Users\User\Desktop\hong\ICTS\ICTS-test-2020\{}_predict{}'.format(model_name,dt)
if not os.path.isdir(predict_dir):
    os.mkdir(predict_dir)

def predict_arrays(img_fdata):
#     print('----resize 至128*128-----')    
#     print(img_fdata.shape)
    
    img_list=[]
    for img in range(img_fdata.shape[0]):
        img_list.append(np.asarray(Image.fromarray(np.uint8(img_fdata[img])).resize((IMG_SIZE,IMG_SIZE))))
                        
    img_arr=np.asarray(img_list).astype('float32')/255.0
    img_arr=img_arr.reshape(-1,IMG_SIZE,IMG_SIZE,1)
#     print('依次預測個檔案的結果  img_arr(86,128,128,1) 或是 img_arr(76,128,128,1')
    y_pred=test_model.predict(img_arr)
    
#   大於 0.5 機率值 =1 ，其他為0
    y_pred=np.where(y_pred > 0.5, 1, 0)

#     print('----resize 回原來大小-----')   
    y_pred_list=[]
    for img in range(y_pred.shape[0]):
        temp=np.uint8(y_pred[img])
        temp=temp.reshape(IMG_SIZE,IMG_SIZE)
        re_img=Image.fromarray(temp).resize((img_fdata.shape[1],img_fdata.shape[1]))
        
        y_pred_list.append(np.asarray(re_img))
    
    y_pred_list_arr=np.asarray(y_pred_list)
#     print(y_pred.shape)
#     print(y_pred_list_arr.shape)
    return y_pred_list_arr # 反回一個 nii.gz 檔案的預測label

################# 
# 1.讀取 nii 文件
# 2.取得 Image array
# 3. 調換 array 維度
# 4. 預測一組 nii  predict_arrays ；產生預測arrays的方法
# 5. 還原 array 維度
# 6. 生成 Nifti1Image 物件
# 7. 儲存 Nifti1Image
####################
# 8.將 1~7 的所有檔案壓縮


# 讀取 並產生 nii.gz 檔案
def re_test_flie(dirPath):
    filenames = os.listdir(dirPath)  #讀取 nii 資料夹
    slice_trans = []
 
    for f in filenames:
        # 讀取 nii 文件
        img_path = os.path.join(dirPath, f)
        img = nib.load(img_path)                #讀取 nii 文件
        img_fdata = img.get_fdata()
        
#         print('img_fdata shape',img_fdata.shape)
        
        img_fdata=img_fdata.transpose((2,1,0))
        
#         print('img_fdata shape',img_fdata.shape)
        
        predict_data=predict_arrays(img_fdata[:,:,:])
        
        
        predict_data=predict_data.transpose((2,1,0))
        new_image = nib.Nifti1Image(predict_data, affine=img.affine)

#         print(predict_file)

        predict_file=r'{}\{}'.format(predict_dir,f)
#         print(predict_file)
        nib.nifti1.save(new_image, predict_file)
    
#   zip 檔案
    zip_file = predict_dir+'.zip'
    if not os.path.exists(predict_dir):                # 如果目录路径不存在
        print('目录不存在')
    else:                                           # 否则压缩目录
        zip = zipfile.ZipFile(zip_file, mode='w', compression=zipfile.ZIP_DEFLATED)
        for item in os.listdir(predict_dir):
            new_file_path = os.path.join(predict_dir,item)
            zip.write(new_file_path,item)
        zip.close()
        
        
test_file_dir=r'C:\Users\User\Desktop\hong\ICTS\ICTS-test-2020\image'
re_test_flie(test_file_dir)

In [ ]:
img = nib.load(r'C:\Users\User\Desktop\hong\ICTS\ICTS-test-2020\predict06171335\2JA4PBWY.nii.gz')                #讀取 nii 文件
img_fdata = img.get_fdata()
set(img_fdata.flatten())

In [ ]:
img = nib.load(r'C:\Users\User\Desktop\hong\ICTS\ICTS-test-2020\predict06170307\2JA4PBWY.nii.gz')                #讀取 nii 文件
img_fdata = img.get_fdata()
set(img_fdata.flatten())

# 測試功能

In [ ]:
zip_file = r'C:\Users\User\Desktop\hong\ICTS\ICTS-test-2020\predict06161801.zip'
if not os.path.exists(predict_dir):                # 如果目录路径不存在
    print('目录不存在')
else:                                           # 否则压缩目录
    zip = zipfile.ZipFile(zip_file, mode='w', compression=zipfile.ZIP_DEFLATED)
    for item in os.listdir(predict_dir):
        new_file_path = os.path.join(predict_dir,item)
        zip.write(new_file_path,item)
    zip.close()

In [ ]:
# 檢視 affine 

file_2JA4PBWY=r'C:\Users\User\Desktop\hong\ICTS\ICTS-test-2020\image\2JA4PBWY.nii.gz'
file_2JA4PBWY=nib.load(file_2JA4PBWY)
print(file_2JA4PBWY.affine)
print(file_2JA4PBWY.get_fdata().shape)

print('----------------------')

img_2JA4PBWY=r'C:\Users\User\Desktop\hong\ICTS\ICTS-test-2020\predict06170259\2JA4PBWY.nii.gz'
img_2JA4PBWY=nib.load(img_2JA4PBWY)
# temp_data=file_2ABGOEM3.get_fdata()
print(img_2JA4PBWY.affine)
print(img_2JA4PBWY.get_fdata().shape)


In [ ]:
def nii_to_image(dirPath):
    filenames = os.listdir(dirPath)  #读取nii文件夹
    slice_trans = []
 
    for f in filenames:
        #开始读取nii文件
        img_path = os.path.join(dirPath, f)
        img = nib.load(img_path)                #读取nii
        img_fdata = img.get_fdata()
        img_fdata=img_fdata.transpose((2,1,0))
        fname = f.replace('.nii.gz','')            #去掉nii的后缀名

        print(img_fdata.shape)


label_file_name=r'C:\Users\User\Desktop\hong\ICTS\label'
nii_to_image(label_file_name)

# nii convert png and move file

In [ ]:
import numpy as np
import os                #遍历文件夹
import nibabel as nib    #nii格式一般都会用到这个包
import imageio           #转换成图像
 
def nii_to_image(dirPath,outdir):
    filenames = os.listdir(dirPath)  #读取nii文件夹
    slice_trans = []
 
    for f in filenames:
        #开始读取nii文件
        img_path = os.path.join(dirPath, f)
        img = nib.load(img_path)                #读取nii
        img_fdata = img.get_fdata()
        img_fdata=img_fdata.transpose((2,1,0))
        fname = f.replace('.nii.gz','')            #去掉nii的后缀名
        img_f_path = os.path.join(outdir)
#         print(img_fdata.shape)
#         print(img_path)
#         print(img_f_path)
#         i=0
#         print(os.path.join(img_f_path,'{}_{}.png'.format(fname,i)))
        #开始转换为图像
        (x,y,z) = img_fdata.shape
        for i in range(0,x,1):                      #z是图像的序列
            silce = img_fdata[i, :, :]          #选择哪个方向的切片都可以
            imageio.imwrite(os.path.join(img_f_path,'{}_{}.png'.format(fname,i)), silce)
            
image_file_name=r'C:\Users\User\Desktop\hong\ICTS\image'
image_file_out=r'C:\Users\User\Desktop\hong\ICTS\img'
nii_to_image(image_file_name,image_file_out)

In [ ]:
label_file_name=r'C:\Users\User\Desktop\hong\ICTS\label'
label_file_out=r'C:\Users\User\Desktop\hong\ICTS\lab'
nii_to_image(label_file_name,label_file_out)

# ICTS-test-2020 convert

In [ ]:
test_file_name=r'C:\Users\User\Desktop\hong\ICTS\ICTS-test-2020\image'
test_file_out=r'C:\Users\User\Desktop\hong\ICTS\ICTS-test-2020\img'
nii_to_image(test_file_name,test_file_out)

# 複製到 train  lab 目錄下 有白色目標的 png ，對應 train img 目錄

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
from PIL import Image

img_file_dir=r'C:\Users\User\Desktop\hong\ICTS\img'
lab_file_dir=r'C:\Users\User\Desktop\hong\ICTS\lab'

img_train_dir=r'C:\Users\User\Desktop\hong\ICTS\train\img'
lab_train_dir=r'C:\Users\User\Desktop\hong\ICTS\train\lab'

def ReFileName(dirPath):
    listdir=os.listdir(dirPath)
    for file in listdir:
        if os.path.isfile(os.path.join(dirPath, file)) == True:
            c= os.path.basename(file)
            name = dirPath + '\\' + c
            image=Image.open(name)
            image_arr=np.asarray(image)
            image_arr=image_arr.flatten()
            
            
            img_name=name.replace('\\lab\\','\\img\\')
            target_lab=name.replace('\\lab\\','\\train\\lab\\')
            target_img=name.replace('\\lab\\','\\train\\img\\')
#             print(name )
#             print(img_name)    
#             print (target_lab)   
#             print  (target_img)  
           
            
            if len(set(image_arr))>1:
                shutil.copy(name,target_lab) 
                shutil.copy(img_name,target_img) 
#                 print(img_name)
#                 print(name)    
#                 os.remove(img_name)
#                 os.remove(name)
#                 print(set(image_arr))
#                 print(len(set(image_arr)))
#                 print('---------------------------')

# ReFileName(lab_file_dir)
